In [ ]:
import pandas as pd
import numpy as np
import os
import sys
sys.path.insert(0,'\\'.join(sys.path[0].split('\\')[:-1]))
from src.config import BASE_DIR

main_dir = os.path.abspath(os.path.join(BASE_DIR,'..'))
project_dir = BASE_DIR


from src.data.load_raw import load_all, load_mensual_data
from src.data.compiling_raw_files import compile_all
from src.data.merge_sources import merge_all
from src.data.standard_simple_merged import clean_standard
from src.client.csin_client import CSIN_Client

import warnings
warnings.filterwarnings("ignore")


In [ ]:
parent_directory = os.path.abspath(os.path.join(os.path.dirname(os.path.abspath(os.getcLF())),'')) # For adding the project URI as a Python PATH.

downloader = CSIN_Client()


month_df = pd.read_csv(os.path.join(parent_directory,'utils','mensual_performance_datalist.csv'), index_col=0)
actuals_m = pd.read_parquet(os.path.join(parent_directory,'compiled_outputs','merged','CSIN_mensual.parquet'))
actuals_m = actuals_m[(actuals_m['reporte'] == 'Plantas_Desempeno') & (actuals_m['date'] >= '2024-01-01')]
data_collected = []
files = actuals_m['filename'].str.split('|', expand=True).copy()
for col in files:
    data_collected.append(files[col].dropna().apply(lambda x: '_'.join(x.split('_')[2:-1])).unique().tolist())
data_collected = sum(data_collected,[])
missing_files = [x for x in month_df['id'].tolist() if x not in data_collected]

month_df = month_df[month_df['id'].isin(missing_files)]


In [ ]:
csrt_token = downloader.csrt_token
import datetime
import requests
from bs4 import BeautifulSoup

session = requests.Session()

plantas_mensual_list = "https://coess.io/app/reportes/mensualperformance/MensualDesempeno"

payload = {
    'pageNumber' : '0',
    'pageSize' : '100000',
    'fromMonthYear' : '2024-01-01',
    'toMonthYear' : datetime.date.today().strftime('%Y-%m-%d'),
    'usuarios' : '',
    'objetoId' : '',
    'csrt' : csrt_token
}

headers = {
            'Authorization': f'Bearer {downloader.jwt_token}',
            'X-Security-Csrf-Token' : downloader.csrf_token
        }

response_list = session.get(plantas_mensual_list, params=payload, headers=headers, verify=False)
soup_list = BeautifulSoup(response_list.text, 'html.parser')

In [ ]:

summary_page = 1
input_size = 2

aux = pd.read_html(soup_list.find('table').prettify())
num_of_inputs = len(aux[summary_page][0].str.split('  ', expand=True))
total_inputs = num_of_inputs*input_size
start = summary_page + 1

rows_base = [x for x in range(start, start+total_inputs, input_size)]

summary = pd.DataFrame()

for row in rows_base:
    tempo = aux[row].T
    tempo.columns = tempo.loc[0,:]
    tempo = tempo.drop(index=0).drop(columns=['submittedBy', 'assesmentDueDate'])
    summary = pd.concat([summary,tempo]).reset_index(drop=True)

